In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString


In [ ]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

# Base directory where all gtfs folders live
base_dir = "/Users/ayemaq/Desktop/Datathon"

# list of gtfs directoiesr
gtfs_dirs = ["gtfs_bx", "gtfs_q", "gtfs_si", "gtfs_m", "gtfs_b"]

# Load NTA shapefile
nta = gpd.read_file("/Users/ayemaq/Desktop/Datathon/nynta2020_25c/nynta2020.shp")
nta = nta.to_crs(epsg=4326)

all_results = []

for gtfs_dir in gtfs_dirs:
    folder_path = os.path.join(base_dir, gtfs_dir)
    print(f"Processing {gtfs_dir}...")
    
    # Load GTFS core files
    shapes = pd.read_csv(os.path.join(folder_path, "shapes.txt"))
    routes = pd.read_csv(os.path.join(folder_path, "routes.txt"))
    trips = pd.read_csv(os.path.join(folder_path, "trips.txt"))
    
    # Convert shapes into LineStrings
    shape_geoms = (
        shapes.sort_values(["shape_id", "shape_pt_sequence"])
        .groupby("shape_id")
        .apply(lambda x: LineString(zip(x.shape_pt_lon, x.shape_pt_lat)))
        .reset_index(name="geometry")
    )
    shapes_gdf = gpd.GeoDataFrame(shape_geoms, geometry="geometry", crs="EPSG:4326")
    
    # Link shape -> trip -> route
    trip_shapes = trips[["route_id", "shape_id"]].drop_duplicates()
    routes_shapes = trip_shapes.merge(routes[["route_id", "route_short_name"]], on="route_id")
    routes_shapes = routes_shapes.merge(shapes_gdf, on="shape_id")
    
    bus_routes_gdf = gpd.GeoDataFrame(routes_shapes, geometry="geometry", crs="EPSG:4326")
    
    # Spatial join with NTA boundaries
    joined = gpd.sjoin(bus_routes_gdf, nta, how="inner", predicate="intersects")
    
    # Keep relevant fields
    result = joined[["route_short_name", "NTA2020", "NTAName", "BoroName"]].drop_duplicates()
    result["borough_file"] = gtfs_dir  # track source
    
    all_results.append(result)

# Combine all boroughs
final_df = pd.concat(all_results, ignore_index=True)
final_df.to_csv(os.path.join(base_dir, "bus_routes_to_nta.csv"), index=False)

print("✅ Done! Output saved as bus_routes_to_nta.csv")

Processing gtfs_bx...


/var/folders/v2/9k8drrwj5h39kpr935vvx6j80000gn/T/ipykernel_26776/2761504567.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: LineString(zip(x.shape_pt_lon, x.shape_pt_lat)))


Processing gtfs_q...


/var/folders/v2/9k8drrwj5h39kpr935vvx6j80000gn/T/ipykernel_26776/2761504567.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: LineString(zip(x.shape_pt_lon, x.shape_pt_lat)))


Processing gtfs_si...


/var/folders/v2/9k8drrwj5h39kpr935vvx6j80000gn/T/ipykernel_26776/2761504567.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: LineString(zip(x.shape_pt_lon, x.shape_pt_lat)))
/var/folders/v2/9k8drrwj5h39kpr935vvx6j80000gn/T/ipykernel_26776/2761504567.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: LineString(zip(x.shape_pt_lon, x.shape_pt_lat)))


Processing gtfs_m...
Processing gtfs_b...


/var/folders/v2/9k8drrwj5h39kpr935vvx6j80000gn/T/ipykernel_26776/2761504567.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: LineString(zip(x.shape_pt_lon, x.shape_pt_lat)))


✅ Done! Output saved as bus_routes_to_nta.csv


In [3]:
import pandas as pd

# Load the new file
df = pd.read_csv("/Users/ayemaq/Desktop/Datathon/bus_routes_to_nta.csv")

# Peek at the data
print(df.tail())

# See how many unique routes
print("Unique bus routes:", df['route_short_name'].nunique())

# See how many NTAs are covered
print("Unique NTAs:", df['NTA2020'].nunique())


     route_short_name NTA2020                   NTAName  BoroName borough_file
2430               B6  BK1103          Gravesend (West)  Brooklyn       gtfs_b
2431               B6  BK1204   Mapleton-Midwood (West)  Brooklyn       gtfs_b
2432               B6  BK0503    East New York-New Lots  Brooklyn       gtfs_b
2433              B41  BK1891  Marine Park-Plumb Island  Brooklyn       gtfs_b
2434              B41  BK1502                   Madison  Brooklyn       gtfs_b
Unique bus routes: 253
Unique NTAs: 244


In [ ]:
poverty = pd.read_csv("/Users/ayemaq/Desktop/Datathon/nta_poverty.csv")
poverty.head()

# Summary stats
print(poverty["avg_poverty_rate"].describe())
# Check max values
print("Max poverty rate:", poverty["avg_poverty_rate"].max())

print(poverty[["NTANAME", "avg_poverty_rate"]].sort_values("avg_poverty_rate", ascending=False).head(10))


   BORONAME NTACODE                  NTANAME  avg_poverty_rate  \
0     Bronx  BX0601               West Farms             1.965   
1    Queens  QN0161  Sunnyside Yards (North)             1.805   
2     Bronx  BX0603                  Belmont             1.785   
3     Bronx  BX0101   Mott Haven-Port Morris             1.747   
4  Brooklyn  BK0103       South Williamsburg             1.738   

   avg_median_income  total_population  
0            32027.0             17898  
1                0.0                44  
2            31272.0             36644  
3            30757.0             56669  
4            39131.0             46005  
count    180.000000
mean       0.694217
std        0.427974
min        0.000000
25%        0.374250
50%        0.605000
75%        0.989750
max        1.965000
Name: avg_poverty_rate, dtype: float64
Max poverty rate: 1.965
                              NTANAME  avg_poverty_rate
0                          West Farms             1.965
1             Sunnysid